In [1]:
import utils
import mujoco
import os
import pickle
from scipy.io import savemat 
from dm_control import mjcf
import numpy as np
import jax
from jax import numpy as jnp
import time
from controller import *
import stac_base

# jax.disable_jit(disable=True)

In [2]:
# If your machine is low on ram:
# os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.6'
# os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false"

def get_clip(kp_data, n_frames):
    import random
    max_index = kp_data.shape[0] - n_frames + 1
    rand_start = random.randint(0, max_index)
    return kp_data[rand_start:rand_start+n_frames,:]

In [3]:
param_path = "../params/params.yaml"
utils.init_params(param_path)

In [4]:
rat_xml = "../models/rodent_stac.xml"
rat23 = "../models/rat23.mat"
data_path = "../save_data_AVG.mat"
# data_path = "/n/holylabs/LABS/olveczky_lab/holylfs02/Everyone/dannce_rig/dannce_ephys/art/2020_12_22_1/DANNCE/predict03/save_data_AVG.mat" 

fit_path = "floating_fit.p"
transform_path = "floating_transform.p"
utils.params['Q_TOL'] = 1e-8
utils.params['n_fit_frames'] = 500
utils.params['N_ITERS'] = 1
skip_transform = True

In [5]:
model = mujoco.MjModel.from_xml_path(rat_xml)
model.opt.solver = mujoco.mjtSolver.mjSOL_NEWTON
model.opt.disableflags = mujoco.mjtDisableBit.mjDSBL_EULERDAMP
model.opt.iterations = 1
model.opt.ls_iterations = 4

In [6]:
start_time = time.time()

utils.params["ROOT_MAXITER"] = 1000
root = mjcf.from_path(rat_xml)

# Default ordering of mj sites is alphabetical, so we reorder to match
kp_names = utils.loadmat(rat23)["joint_names"]
utils.params["kp_names"] = kp_names

# argsort returns the indices that would sort the array
stac_keypoint_order = np.argsort(kp_names)
# Load kp_data, /1000 to scale data (from mm to meters i think?)
kp_data = utils.loadmat(data_path)["pred"][:] / 1000

kp_data = prep_kp_data(kp_data, stac_keypoint_order)

# setup for fit
physics, mj_model = set_body_sites(root)
part_opt_setup(physics)

# Run root optimization
# Create mjx model and data
mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.make_data(mjx_model)
# Get and set the offsets of the markers
offsets = jnp.copy(stac_base.get_site_pos(mjx_model))
offsets *= utils.params['SCALE_FACTOR']

# print(mjx_model.site_pos, mjx_model.site_pos.shape)
mjx_model = stac_base.set_site_pos(mjx_model, offsets)

# forward is used to calculate xpos and such
mjx_data = mjx.kinematics(mjx_model, mjx_data)
mjx_data = mjx.com_pos(mjx_model, mjx_data)
print(f"tol = {utils.params["Q_TOL"]}")
mjx_data = root_optimization(mjx_model, mjx_data, kp_data)

Root Optimization:
lb: (74,)
 ub: (74,)


2024-02-20 14:12:56.147790: E external/xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit_q_opt] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2024-02-20 14:15:48.041300: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 4m51.89963s

********************************
[Compiling module jit_q_opt] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


q_opt 1 finished in 390.6724708080292 with an error of 1.9894706992928946e-09
Replace 1 finished in 317.7037088871002
starting q_opt 2
starting qs: [ 0.34545239  0.0708037   0.03804783  0.84471372  0.06146757 -0.06952961
  0.5271111   0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
q_opt 1 finished in 0.0073649883270263

2024-02-20 14:21:05.912950: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 5m2.867534s

********************************
[Compiling module jit_kinematics] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


save in the same format and use viz.py

In [7]:
offset_path = "root.p"
data = {
        "kp_data": kp_data,
        "qpos": [mjx_data.qpos[:]],
        "offsets": offsets,
        # "walker_body_sites": [stac_base.get_site_xpos(mjx_data)],
        "xpos": [mjx_data.xpos[:]],
        "names_qpos": initialize_part_names(physics) # utils.params["part_names"],
        # "names_xpos": physics.named.data.xpos.axes.row.names,
    }
if os.path.dirname(offset_path) != "":
    os.makedirs(os.path.dirname(offset_path), exist_ok=True)
with open(offset_path, "wb") as output_file:
    pickle.dump(data, output_file, protocol=2)